### simulation - with transaction fee, taxes / without dividends

# 초기 조건 / 데이터 임포트 / 벤치마크 계산

## 라이브러리 임포트 / 초기 조건 설정

In [292]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [293]:
import pandas as pd
import numpy as np
from datetime import datetime
import math

In [294]:
transaction_fee_rate = 0.0025
tax_rate = 0.22
tax_threshold = 2500000 / 1100
initial_cash = 10000

In [295]:
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

In [296]:
strategies = []
returns = []

## 데이터 임포트

In [297]:
df = pd.read_csv('/content/drive/MyDrive/quant_projects/momentum_strategies/SPY.csv')[:-2] # 2021-09까지 결과 출력
closes = df.Close.to_numpy()
opens = df.Open.to_numpy()
df['Date'] = df['Date'].map(lambda x: x[:-3])

In [298]:
tb3_df = pd.read_csv('/content/drive/MyDrive/quant_projects/momentum_strategies/TB3MS.csv')
risk_free_rates = tb3_df.TB3MS.to_numpy() / 100

In [299]:
dividend_df = pd.read_csv('/content/drive/MyDrive/quant_projects/momentum_strategies/SPY_dividends.csv')
dividend_df['Date'] = dividend_df['Date'].map(lambda x: x[:-3])
dividend_df.sort_values(by=['Date'], inplace=True)

In [300]:
dividend_df['Month'] = dividend_df['Date'].map(lambda x: df.index[df['Date']==x].item())

In [301]:
dividends = np.zeros(len(closes))
# 배당이 없는 달은 0으로 처리
for month, div in zip(dividend_df.Month, dividend_df.Dividends.map(lambda x: round(x,3))):
  dividends[month] = div

## mdd / statistics

In [302]:
# 출처: http://blog.quantylab.com/mdd.html
def get_mdd(x):
    """
    MDD(Maximum Draw-Down)
    :return: (peak_upper, peak_lower, mdd rate)
    """
    arr_v = np.array(x)
    peak_lower = np.argmax(np.maximum.accumulate(arr_v) - arr_v)
    if peak_lower == 0:
      peak_upper = 0
    else:
      peak_upper = np.argmax(arr_v[:peak_lower])
    return peak_upper, peak_lower, (arr_v[peak_lower] - arr_v[peak_upper]) / arr_v[peak_upper]

In [303]:
# highs = df.High.to_numpy()
# lows = df.Low.to_numpy()

In [304]:
# np.argmax(np.maximum.accumulate(highs))

In [305]:
# 월중 mdd 
# max((np.maximum.accumulate(highs) - lows) / np.maximum.accumulate(highs))

In [306]:
# 월중 mdd
# peak_lower = np.argmax((np.maximum.accumulate(highs) - lows) / np.maximum.accumulate(highs))
# peak_upper = np.argmax(highs[:peak_lower])
# print(peak_upper, peak_lower, (lows[peak_lower] - highs[peak_upper]) / highs[peak_upper])

In [307]:
# get_mdd(benchmark)

In [308]:
def annualize(closes, opens, month_lag):
  annual_returns = []
  for i in range(len(closes) // 12 +1):
    if (i+1)*12 > len(closes):
      annual_returns.append(round((closes[-1] - opens[i*12-month_lag]) / opens[i*12-month_lag],4))
    elif i*12-month_lag < 0:
      annual_returns.append(round((closes[i*12+11-month_lag] - opens[0]) / opens[0], 4))
    else:
      annual_returns.append(round((closes[i*12+11-month_lag] - opens[i*12-month_lag]) / opens[i*12-month_lag],4))
  return annual_returns

In [309]:
def statistics(name, closes, annual_returns, current_flag=False):
  mean_risk_free_rate = np.mean(risk_free_rates) if current_flag else 0.005
  cagr = round(pow(closes[-1] / closes[0], 1/ (round(len(closes) / 12))) - 1, 4)
  mean_annual = round(np.mean(annual_returns),4)
  stddev = round(np.std(annual_returns),4)
  sharpe = round((np.mean(annual_returns)- mean_risk_free_rate) / np.std(annual_returns),4)
  worst_year = round(min(annual_returns), 4)
  best_year = round(max(annual_returns), 4)
  mdd = round(get_mdd(closes)[2], 4)
  return {'name':name, 'initial_balance': round(closes[0], 2), 'final_balance':round(closes[-1],2), 'cagr':cagr, 'mean_annual':mean_annual, 'stddev':stddev, 'sharpe':sharpe, 'worst_year':worst_year, 'best_year': best_year, 'mdd': mdd}

## 벤치마크 계산

In [310]:
benchmark_shares = initial_cash // closes[0]
benchmark = closes * benchmark_shares

In [311]:
strategies.append({**statistics('Benchmark', benchmark, annualize(closes, opens, 1)), 'accum_fees':0, 'accum_taxes':0})
returns.append({"name":'Benchmark', "df":benchmark})

In [312]:
strategies

[{'accum_fees': 0,
  'accum_taxes': 0,
  'best_year': 0.3453,
  'cagr': 0.0814,
  'final_balance': 96556.5,
  'initial_balance': 9991.41,
  'mdd': -0.522,
  'mean_annual': 0.0922,
  'name': 'Benchmark',
  'sharpe': 0.5243,
  'stddev': 0.1664,
  'worst_year': -0.3842}]

# Backtesting

In [321]:
class Portfolio:
  transaction_fee_rate = 0.0025
  tax_rate = 0.22
  tax_threshold = 2500000 / 1100

  def __init__(self, benchmark_df, risk_free_rates, portfolio_details):
    self.benchmark_opens = benchmark_df.Open.to_numpy()
    self.benchmark_closes = benchmark_df.Close.to_numpy()
    self.benchmark_highs = benchmark_df.High.to_numpy()
    self.benchmark_lows = benchmark_df.Low.to_numpy()
    self.benchmark_dates = benchmark_df.Date.to_numpy()
    self.risk_free_rates = risk_free_rates
    self.columns = ["upwards", "cash", "investments", "shares", "buy_prices", "fees", "accum_fees", "dividend_incomes", "month_gains", "annual_gains", "annual_taxes", "accum_taxes", "portfolio_closes", "portfolio_opens", "portfolio_highs", "portfolio_lows"]
    for column in columns:
      setattr(self, column, np.zeros(len(self.benchmark_dates)))
    for key, value in portfolio_details.items():
      setattr(self, key, value)
    self.df = pd.DataFrame({'date': self.benchmark_dates, 'opens': self.benchmark_opens, 'closes': self.benchmark_closes, 'highs':self.benchmark_highs, 'lows':self.benchmark_lows})
    self.name = f"{self.benchmark_name}_{self.rebalancing_period}:{self.look_back}_{self.start_month}to{self.start_month + self.rebalancing_period}ex{self.exclude_period}{'_tax' if self.apply_tax else ''}{'_fee' if self.apply_fee else ''}{'_div' if len(dividends) else ''}"

  def buy(self, i, buy_shares):
    self.shares[i] += buy_shares
    buy_amount = buy_shares * self.benchmark_closes[i]
    self.investments[i] += buy_amount
    self.cash[i] -= buy_amount * (1 + transaction_fee_rate)
    if self.apply_fee:
      self.fees[i] += buy_amount * transaction_fee_rate
    self.buy_prices[i] = (self.buy_prices[i-1] * self.shares[i-1] + buy_amount) / self.shares[i] 

  def sell(self, i, sell_shares):
    self.shares[i] -= sell_shares
    sell_amount = sell_shares * self.benchmark_closes[i]
    self.investments[i] -= sell_amount
    self.cash[i] += sell_amount * (1 - transaction_fee_rate)
    if self.apply_fee:
      self.fees[i] += sell_amount * transaction_fee_rate
    self.buy_prices[i] = 0 if self.shares[i] == 0 else self.buy_prices[i-1]
    self.month_gains[i] += self.sell_trade_gains(i, sell_shares)

  def sell_trade_gains(self, i, sell_shares):
    # returns expected_trade_gains (양도수익 - 매매대금 수수료)
    return (sell_shares * (self.benchmark_closes[i] - self.buy_prices[i-1]) - sell_shares * self.benchmark_closes[i] * transaction_fee_rate)

  def pay_tax(self, i, current_tax):
    # 올해 세금 확정 후 납부
    self.annual_taxes[i] = current_tax
    self.cash[i] -= current_tax

  def capital_gains_tax(self, i):
    self.annual_gains[i] = sum(self.month_gains[i-11:i+1])
    if self.annual_gains[i] > tax_threshold:
      # 1)세금을 내야 하는 경우
      current_tax = (self.annual_gains[i] - tax_threshold) * tax_rate
      if self.cash[i] >= current_tax:
        # 1-1)보유현금이 세금보다 많은 경우
        self.pay_tax(i, current_tax)
      else:
        # 1-2)추가로 매도하여 세금을 내야하는 경우
        shares_minus = math.ceil((current_tax - self.cash[i]) / (1 - transaction_fee_rate) / self.benchmark_closes[i])
        trade_gains = self.sell_trade_gains(i, shares_minus)
        if (self.annual_gains[i] + trade_gains > tax_threshold):
          # 1-2-A)세금을 내기 위해 매도했는데 1-2-A1)확정 수익이 추가로 발생하거나 / 1-2-A2)확정 손실이 작아 세금을 내야 하는 경우
          if trade_gains > 0:
            # 1-2-A1)확정 수익이 추가로 발생 -> 추가분까지 고려하여 매도수량 계산
            shares_minus = math.ceil(((self.annual_gains[i] - tax_threshold) * tax_rate - self.cash[i]) / (1 - transaction_fee_rate) / (self.benchmark_closes[i] * (1 - tax_rate) + self.buy_prices[i-1] * tax_rate))
            trade_gains = self.sell_trade_gains(i, shares_minus)
          # 매도 후 양도수익 업데이트
          self.sell(i, shares_minus)
          self.annual_gains[i] += trade_gains
          # 세금 업데이트 후 납부
          current_tax = (self.annual_gains[i] - tax_threshold) * tax_rate
          self.pay_tax(i, current_tax)
        else:
          # 1-2-B)세금을 내기 위해 매도했는데 세금보다 확정 손실이 커서 세금을 내지 않아도 되는 경우
          # 매도 후 양도수익 업데이트
          self.sell(i, shares_minus)
          self.annual_gains[i] += trade_gains

  def save_result(self):
    for column in self.columns:
      self.df[column] = getattr(self, column).tolist()
    # self.df['benchmark'] = self.benchmark.tolist()
    # self.df.to_csv(f'/content/drive/MyDrive/quant_projects/momentum_strategies/{current_date}_{self.name}.csv')
    
  def init_portfolio(self):
    self.cash[0:self.look_back] = self.initial_cash
    self.portfolio_opens[0:self.look_back] = self.initial_cash
    self.portfolio_closes[0:self.look_back] = self.initial_cash
    self.portfolio_highs[0:self.look_back] = self.initial_cash
    self.portfolio_lows[0:self.look_back] = self.initial_cash
  
  def intra_month(self, i):
      current_dividend = dividends[i] * self.shares[i-1]
      self.cash[i] = self.cash[i-1] + current_dividend
      self.dividend_incomes[i] = current_dividend
      self.shares[i] = self.shares[i-1]
      self.portfolio_opens[i] = self.shares[i] * self.benchmark_opens[i] + self.cash[i]
      self.portfolio_highs[i] = self.shares[i] * self.benchmark_highs[i] + self.cash[i]
      self.portfolio_lows[i] = self.shares[i] * self.benchmark_lows[i] + self.cash[i]
      self.investments[i] = self.shares[i] * self.benchmark_closes[i]
      self.buy_prices[i] = self.buy_prices[i-1]

  def trading(self, i, trade_month_index):
    # trade with upwards (signal)
    if i % self.rebalancing_period == trade_month_index:
      self.upwards[i] = 1 if (self.benchmark_closes[i-self.exclude_period] - self.benchmark_closes[i-self.look_back]) / self.benchmark_closes[i-self.look_back] >= self.risk_free_rates[i] else 0
      previus_i = i - self.rebalancing_period
      while previus_i < 0:
        previus_i += 1
      if self.upwards[i] == 1:
        # 0 - 1 (신규 매수) / 1 - 1 (추가 매수)
        shares_plus = self.cash[i] / (1 + transaction_fee_rate) // self.benchmark_closes[i]
        self.buy(i, shares_plus)
      elif self.upwards[previus_i] == 1 and self.upwards[i] == 0:
        # 1 - 0 (전량 매도)
        self.sell(i, self.shares[i])

  def backtest(self):
    trade_month_index = (self.start_month - int(self.benchmark_dates[0][-2:])) % self.rebalancing_period
    while trade_month_index < 0:
      # ex) self.rebalancing_period == 1, trade_month_index == -7 -> trade_month_index becomes 0 
      # ex) self.rebalancing_period == 6, trade_month_index == -1 -> trade_month_index becomes 5 
      trade_month_index += self.rebalancing_period 
    tax_month_index = (12 - int(self.benchmark_dates[0][-2:])) % 12
    self.init_portfolio()
    # self.look_back 시점 이후 거래 시작
    for i in range(self.look_back, len(self.benchmark_dates)):
      # 월말 거래 전
      self.intra_month(i)
      # trade
      self.trading(i, trade_month_index)
      self.portfolio_closes[i] = self.shares[i] * self.benchmark_closes[i] + self.cash[i]
      # tax
      if self.apply_tax and (i % 12 == tax_month_index):
        self.capital_gains_tax(i)
      self.accum_taxes[i] = self.accum_taxes[i-1] + self.annual_taxes[i]
      # fees
      self.accum_fees[i] = self.accum_fees[i-1] + self.fees[i]
    self.save_result()
    strategies.append(self.statistics())
    returns.append({"name":self.name, "df":self.df['portfolio_closes'].to_numpy()})

  def mdd(self):
    peak_lower = np.argmax(np.maximum.accumulate(self.portfolio_closes) - self.portfolio_closes)
    if peak_lower == 0:
      peak_upper = 0
    else:
      peak_upper = np.argmax(self.portfolio_closes[:peak_lower])
    # 월중 mdd
    # peak_lower = np.argmax((np.maximum.accumulate(self.portfolio_highs) - self.portfolio_lows) / np.maximum.accumulate(self.portfolio_highs))
    # peak_upper = np.argmax(self.portfolio_highs[:peak_lower])
    # print(peak_upper, peak_lower, (self.portfolio_lows[peak_lower] - self.portfolio_highs[peak_upper]) / self.portfolio_highs[peak_upper])
    return (self.portfolio_closes[peak_lower] - self.portfolio_closes[peak_upper]) / self.portfolio_closes[peak_upper]

  def annual_returns(self):
    month_lag = int(self.benchmark_dates[0][-2:]) - 1
    annual_returns = []
    for i in range(len(self.portfolio_closes) // 12 +1):
      if (i+1)*12 > len(self.portfolio_closes):
        annual_returns.append(round((self.portfolio_closes[-1] - self.portfolio_opens[i*12-month_lag]) / self.portfolio_opens[i*12-month_lag],4))
      elif i*12-month_lag < 0:
        annual_returns.append(round((self.portfolio_closes[i*12+11-month_lag] - self.portfolio_opens[0]) / self.portfolio_opens[0], 4))
      else:
        annual_returns.append(round((self.portfolio_closes[i*12+11-month_lag] - self.portfolio_opens[i*12-month_lag]) / self.portfolio_opens[i*12-month_lag],4))
    return annual_returns

  def monthly_returns(self):
    return[round((self.portfolio_closes[i] - self.portfolio_opens[i]) / self.portfolio_opens[i], 4) for i in range(len(self.benchmark_dates))]

  def statistics(self, is_annual=True, is_average=False):
    time_series_returns = self.annual_returns() if is_annual else self.monthly_returns()
    mean_risk_free_rate = np.mean(self.risk_free_rates) if is_average else 0.005
    cagr = round(pow(self.portfolio_closes[-1] / self.portfolio_closes[0], 1 / len(time_series_returns)) - 1, 4)
    mean_return = round(np.mean(time_series_returns),4)
    stddev = round(np.std(time_series_returns),4)
    sharpe = round((np.mean(time_series_returns)- mean_risk_free_rate) / np.std(time_series_returns),4)
    worst_time_series = round(min(time_series_returns), 4)
    best_time_series = round(max(time_series_returns), 4)
    mdd = round(self.mdd(), 4)
    return {
          'name':self.name, 
          'initial_balance': round(self.portfolio_closes[0], 2), 
          'final_balance':round(self.portfolio_closes[-1],2), 
          'cagr_annual':cagr, 
          'mean_annual':mean_return, 
          'stddev':stddev, 
          'sharpe':sharpe, 
          'worst_year':worst_time_series, 
          'best_year': best_time_series, 
          'mdd': mdd, 
          'accum_fees':round(self.accum_fees[-1], 2), 
          'accum_taxes':round(self.accum_taxes[-1], 2)
        } if is_annual else {
          'name':self.name, 
          'initial_balance': round(self.portfolio_closes[0], 2), 
          'final_balance':round(self.portfolio_closes[-1],2), 
          'cagr_monthly':cagr, 
          'mean_monthly':mean_return, 
          'stddev':stddev, 
          'sharpe':sharpe, 
          'worst_month':worst_time_series, 
          'best_month': best_time_series, 
          'mdd': mdd, 
          'accum_fees':round(self.accum_fees[-1], 2), 
          'accum_taxes':round(self.accum_taxes[-1], 2)
        }

In [281]:
# 단일 건 테스트
portfolio_details = {"initial_cash":10000, "benchmark_name":"SPY", "rebalancing_period":6,"look_back":12, "exclude_period":0, "start_month":4,"apply_tax":True, "apply_fee":True, "benchmark_dividends":dividends}
portfolio = Portfolio(df, risk_free_rates, portfolio_details)
portfolio.backtest()

In [322]:
rebalancing_periods = [1,3,6,12]
look_backs = [1,3,6,12]
for rebalancing_period in rebalancing_periods:
  for look_back in look_backs:
    for index in range(rebalancing_period):
      for exclude_period in range(min(3, look_back)):
        portfolio_details = {"initial_cash":10000, "benchmark_name":"SPY", "rebalancing_period":rebalancing_period,"look_back":look_back, "exclude_period":exclude_period, "start_month":2+index,"apply_tax":True, "apply_fee":True, "benchmark_dividends":dividends}
        portfolio = Portfolio(df, risk_free_rates, portfolio_details)
        portfolio.backtest()

# 결과 출력

## statistics / returns csv 출력

In [323]:
statistics_df = pd.DataFrame(strategies)
# print(statistics_df)
# print(statistics_df.iloc[statistics_df['final_balance'].idxmax()])

In [324]:
statistics_df.sort_values(by=['cagr'], inplace=True, ascending=False)

In [325]:
statistics_df

,name,initial_balance,final_balance,cagr,mean_annual,stddev,sharpe,worst_year,best_year,mdd,accum_fees,accum_taxes,cagr_annual
311,SPY_6:12_4to10ex2_tax_fee_div,10000.0,149886.19,0.0979,0.1105,0.1158,0.9105,-0.1008,0.3307,-0.1936,938.34,12309.67,NaN
413,SPY_12:12_4to16ex2_tax_fee_div,10000.0,135842.56,0.0941,0.1077,0.1213,0.8467,-0.0536,0.3705,-0.1919,783.97,11840.88,NaN
309,SPY_6:12_4to10ex0_tax_fee_div,10000.0,135026.57,0.0939,0.1080,0.1121,0.9184,-0.0539,0.3308,-0.1935,1344.47,18157.35,NaN
417,SPY_12:12_6to18ex0_tax_fee_div,10000.0,133468.41,0.0935,0.1017,0.1254,0.7707,-0.1183,0.3297,-0.1928,307.50,2320.96,NaN
260,SPY_3:12_4to7ex2_tax_fee_div,10000.0,129396.06,0.0923,0.1054,0.1068,0.9396,-0.0991,0.3242,-0.2257,2583.77,14795.94,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,SPY_12:12_12to24ex1_tax_fee_div,10000.0,64260.07,NaN,0.0782,0.1446,0.5063,-0.3667,0.3291,-0.4367,435.17,2050.71,1.0663
217,SPY_12:12_12to24ex2_tax_fee_div,10000.0,43410.36,NaN,0.0653,0.1377,0.4379,-0.3658,0.3232,-0.3543,580.09,4494.55,1.0519
218,SPY_12:12_13to25ex0_tax_fee_div,10000.0,79175.96,NaN,0.0854,0.1108,0.7259,-0.1042,0.3217,-0.2692,825.09,8256.17,1.0740
219,SPY_12:12_13to25ex1_tax_fee_div,10000.0,81472.77,NaN,0.0866,0.1072,0.7618,-0.1042,0.3217,-0.3132,755.08,8721.82,1.0750


In [326]:
statistics_df.to_csv(f'/content/drive/MyDrive/quant_projects/momentum_strategies/{current_date}_statistics.csv')

In [327]:
returns_df = pd.DataFrame({'Date': df['Date']})
for portfolio in returns:
  returns_df[portfolio['name']] = portfolio['df']
print(returns_df)
returns_df.to_csv(f'/content/drive/MyDrive/quant_projects/momentum_strategies/{current_date}_returns.csv')

        Date  ...  SPY_12:12_13to25ex2_tax_fee_div
0    1993-02  ...                     10000.000000
1    1993-03  ...                     10000.000000
2    1993-04  ...                     10000.000000
3    1993-05  ...                     10000.000000
4    1993-06  ...                     10000.000000
..       ...  ...                              ...
339  2021-05  ...                     50866.373006
340  2021-06  ...                     51993.891686
341  2021-07  ...                     53247.893126
342  2021-08  ...                     54813.891686
343  2021-09  ...                     52294.853726

[344 rows x 222 columns]


## 그래프 출력

In [320]:
chart = returns_df.plot.line(x='Date', grid=True, figsize=(40,25), xticks=np.arange(0, 346, 12), yticks=np.arange(0, 150000, 10000))
chart.tick_params(axis='x', rotation=90)
chart.figure.savefig(f'/content/drive/MyDrive/quant_projects/momentum_strategies/{current_date}_returns.jpg')

Output hidden; open in https://colab.research.google.com to view.